<a href="https://colab.research.google.com/github/khush-kantaria/lok-sabha-debates-analysis/blob/main/Scrapper%2Bpdf2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lok Sabha Debates Analysis 
# Scrapper + pdf2csv
 

# Required Imports and Downloads



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_sm

2022-12-02 11:26:36.430960: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32892 sha256=f74e6a44fefa13a003ff4755f585212a7e6378bc901b1b0108231787de6856fc
  Stored in directory: /root/.cache/pip/wheels/75/66/8b/d1acbac7d49f3d98ade76c51ae5d72cec1866131a3b1ad9f82
Successfully built tika


In [ ]:
# imports

import os
import requests
import numpy as np
import multiprocessing
from tika import parser
from bs4 import BeautifulSoup


import re
import pandas as pd
from tqdm import tqdm


import matplotlib.pyplot as plt

# Dataset Preparation

## Web Scrapper

In [ ]:
# a function to scrape the lok sabha digital library and download the pdfs
myvar = 0
stred = ""
def scrape(path):

  # extracting folder name and creating a folder in drive
  folderName = path[208:212]

  if folderName in os.listdir("/content/drive/Shareddrives/NLP_Project/Legislative Data/"):
    print(folderName, " already present")
    
  else:
    folderPath = "/content/drive/Shareddrives/NLP_Project/Legislative Data/" + folderName + "/pdf/"
    folderPathCSV = "/content/drive/Shareddrives/NLP_Project/Legislative Data/" + folderName + "/csv/"
    os.makedirs(folderPath)
    os.makedirs(folderPathCSV)

    url = path
    links = []

    while url != None:
      req = requests.get(url)
      soup = BeautifulSoup(req.content, 'html.parser')
      '''
      # panel_heading = soup.findAll('div',attrs={'class':['panel-info','panel-heading']})
      # stred = str(panel_heading)
      # myvar = 
      # print(np.array(panel_heading))'''
      # trying to fetch all the intermediate urls to the target pdfs
      table = soup.find('table', attrs = {'summary':'This table browses all dspace content'})
      for row in table.findAll('td', attrs = {'headers':'t4'}):
        if(row.a != None):
          link = row.a['href']
          links.append("https://eparlib.nic.in"+link)
      
      url = soup.find('a', attrs = {'class' : 'pull-right'})
      if url != None:
        url = "https://eparlib.nic.in" + url['href']
    
    # get to the pdf from here on
    for number,link in enumerate(links):
      req = requests.get(link)
      soup = BeautifulSoup(req.content, 'html.parser')

      table = soup.find('table', attrs = {'class' : 'table panel-body'})
      for row in table.findAll('td', attrs = {'align':'center'}):
        if(row.a != None):
          pdfLink = row.a['href']
          pdfLink = "https://eparlib.nic.in" + pdfLink

          # extracting and saving the pdfs
          try:
            pdfReq = requests.get(pdfLink)
            pdfName = str(number+1)
            pdfPath = folderPath + pdfName + ".pdf"
            #uncomment below to save files
            with open(pdfPath, 'wb') as f:
              f.write(pdfReq.content)

          except requests.exceptions.HTTPError as errh:
              print("Http Error:",errh)
              print("Number : ", pdfName)
          except requests.exceptions.ConnectionError as errc:
              print("Error Connecting:",errc)
              print("Number : ", pdfName)
          except requests.exceptions.Timeout as errt:
              print("Timeout Error:",errt)
              print("Number : ", pdfName)
          except requests.exceptions.RequestException as err:
              print("OOps: Something Else",err)
              print("Number : ", pdfName)

          

    print("Done with ", folderName)

if __name__ == '__main__':
    
  pool = multiprocessing.Pool()
  pool = multiprocessing.Pool(processes=2)
  year = input("Enter the year: ")


  paths = [
      "https://eparlib.nic.in/handle/123456789/7/simple-search?page-token=0431cfac1e20&page-token-value=afefbd25ee4f0d4f006fe90aa5db2d21&location=123456789%2F7&query=&filtername=date&filtertype=contains&filterquery={year}&rpp=100&sort_by=dc.date_dt&order=desc".format(year=year),
  ]
  pool.map(scrape,paths)

Enter the year: 2002
Done with  2002


## PDF 2 CSV converter

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' '}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
        if s[k]==')':
            while k!=-1 and s[k]!='(':
                k=k-1
        else:
            k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
        k=k+1
    #print(s[i])
    return s[k:]

def Pdf2CsvFolder(folder_path: str):
    pdfs = os.path.join(folder_path, 'pdf')
    csvs = os.path.join(folder_path, 'csv')
    print(pdfs, csvs)
    file_names = os.listdir(pdfs)
    for name in tqdm(file_names):
        Pdf2Csv(os.path.join(pdfs, name), csvs)

def Pdf2Csv(path: str, outDir: str):
    if os.path.basename(path)[:-4] in os.listdir(outDir):
        # print("File", os.path.basename(path)[:-4], "already exists")
        return
    raw = parser.from_file(path)
    content = raw['content'].lower()

    i = 0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
        startindlist.append(match.start())
        i+=1
    
    startindlist.append(len(content))

    convlist = []
    for i in range(len(startindlist)-1):
        convlist.append(content[startindlist[i]-20 : startindlist[i+1]])
    
    for j,i in enumerate(convlist):
        convlist[j] = re.sub('\(Interruptions\)', '', i)
        convlist[j] = re.sub('\(Interruptions\)', '', i)
    
    convlist1 = []

    for s in convlist:
        s = removeend(s)
        s = removestart(s)
        convlist1.append(s)
    
    convlist2=[]
    for i in convlist1:
        convlist2.append(re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i))

    convlist3=[]
    for i in convlist2:
        i = re.sub(r'\d\d\*.\d\d\*.20\d\d', '', i)
        convlist3.append(i)
    
    convlist4=[]
    for i in convlist3:
        convlist4.append(re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i))
    
    convlist5=[]
    for i in convlist4:
        i = re.sub(r'\n', '', i)
        i = re.sub(r' +', ' ', i)
        i = re.sub(r'_+', ' ', i)
        i = re.sub(r'\.+', ' ', i)
        i = re.sub(r'…', ' ', i)
        convlist5.append(i)
    
    for j,i in enumerate(convlist5):
        if len(i)!=0 and i[-1]!='.':
            convlist5[j] = i + '.'

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist5):
        if len(i)!=0:
            convlist6.append(convlist5[j])
            ind+=1
    
    convlist6 = convlist6[:-1]

    df = pd.DataFrame(convlist6)

    df.to_csv(os.path.join(outDir, os.path.basename(path)[:-4]))
    # print("File", os.path.basename(path)[:-4], "done")

In [ ]:
FolderPath = "/content/drive/Shareddrives/NLP_Project/Legislative Data/2002/"

Pdf2CsvFolder(FolderPath)

/content/drive/Shareddrives/NLP_Project/Legislative Data/2002/pdf /content/drive/Shareddrives/NLP_Project/Legislative Data/2002/csv


  0%|          | 0/85 [00:00<?, ?it/s]2022-12-02 11:49:53,826 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2022-12-02 11:49:54,513 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-12-02 11:49:54,980 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
100%|██████████| 85/85 [08:07<00:00,  5.73s/it]
